# Answers - OC Lab 3


In [20]:
# Utility functions

from IPython.display import Markdown, display


def show_code(path):
    with open(path) as f:
        display(Markdown("```" + f.read() + "```"))


base_value = None


def calc_speedup(new_label, new_value):
    global base_value

    if base_value is None:
        raise ValueError("Base value is not set!")

    speedup = base_value / new_value

    display(
        Markdown(
            """$\\text{SpeedUp} = \\frac{\\text{CPU}_{{\\text{Time}}_{\\text{Base}}}}{\\text{CPU}_{{\\text{Time}}_{\\text{%s}}}} = \\frac{\\text{\\# Cycles}_{\\text{Base}}}{\\text{\\# Cycles}_{\\text{%s}}} = \\frac{%d}{%d} \\approx %.3f$

(simulated CPU Rate is consistent)"""
            % (new_label, new_label, base_value, new_value, speedup)
        )
    )


## 2.1


### e)


| **$\text{Clock Cycles}$** | $17$ | **$\text{Instructions}$** | $7$ | **$\text{Average CPI}$** | $\frac{17}{7} \approx 2.429$ |
| ------------------------- | ---- | ------------------------- | --- | ------------------------ | ---------------------------- |


![2.1](assets/2.1.png)


### f)


| **$\text{Clock Cycles}$** | $174$   |
| ------------------------- | ------- |
| **$\text{Instructions}$** | $61$    |
| **$\text{Average CPI}$**  | $2.852$ |

Stalls
| **$\text{Data}$** | $101$ |
| ----------------------------------|--------|
| **$\text{Structural}$** | $0$ |
| **$\text{Branch Taken}$** | $8$ |


In [22]:
base_value = 174  # clock cycles


### g)


By analyzing the program execution, it is clear that at the end of each loop iteration, the simulator loads into the pipeline the `sw $9, mult($0)` instruction (which is the one immediately following the `bne $6, $5, loop` instruction and the first one outside the loop), performing an Instruction Fetch before the branch is resolved and it is known that the loop will be repeated, causing the `sw` to be nullified and a new instruction (the `lw $12, 0($1)` at the beginning of the loop) to be fetched. The exception is, of course, after the last iteration, when the `sw` instruction is indeed the one supposed to run next and is allowed to proceed, therefore having no stalls occur.
As such, it can be concluded that the simulator adopts a Predict Branch Not Taken policy, as it always assumes that the branch will not be taken and starts executing instructions following the `bne` instruction.


## 2.2


### b)


![2.2](assets/2.2.png)


### c)


| **$\text{Clock Cycles}$** | $136$   |
| ------------------------- | ------- |
| **$\text{Instructions}$** | $61$    |
| **$\text{Average CPI}$**  | $2.230$ |

Stalls
| **$\text{Data}$** | $63$ |
| ----------------------------------|--------|
| **$\text{Structural}$** | $9$ |
| **$\text{Branch Taken}$** | $8$ |


### d)


In [23]:
calc_speedup("Forwarding", 136)


$\text{SpeedUp} = \frac{\text{CPU}_{{\text{Time}}_{\text{Base}}}}{\text{CPU}_{{\text{Time}}_{\text{Forwarding}}}} = \frac{\text{\# Cycles}_{\text{Base}}}{\text{\# Cycles}_{\text{Forwarding}}} = \frac{174}{136} \approx 1.279$

(simulated CPU Rate is consistent)

## 2.3


### a)


In [14]:
show_code("lab3_kit/2.3prog.s")


```                .data
A:              .word   1, 3, 1, 6, 4
                .word   2, 4, 3, 9, 5
mult:   .word   0

        .code
        daddi   $1, $0, A      ; *A[0]
        daddi   $5, $0, 1      ; $5 = 1 ;; i    
        daddi   $6, $0, 10     ; $6 = N ;; N = 10
        lw      $9, 0($1)      ; $9 = A[0]  ;; mult
        daddi   $1, $1, 8      ;

loop:   lw      $12, 0($1)     ; $12 = A[i]
        daddi   $5, $5, 1      ; i++
        dmul    $12, $12, $9   ; $12 = $12*$9 ;; $12 = A[i]*mult
        daddi   $1, $1, 8      ; 
        dadd    $9, $9, $12    ; $9 = $9 + $12  ;; mult = mult + A[i]*mult

        bne     $6, $5, loop   ; Exit loop if i == N
        
        sw      $9, mult($0)   ; Store result
        halt

;; Expected result: mult = f6180 (hex), 1008000 (dec)

```

### b)


![2.3](assets/2.3.png)


### c)


| **$\text{Clock Cycles}$** | $118$   |
| ------------------------- | ------- |
| **$\text{Instructions}$** | $61$    |
| **$\text{Average CPI}$**  | $1.934$ |

Stalls
| **$\text{Data}$** | $36$ |
| ----------------------------------|--------|
| **$\text{Structural}$** | $9$ |
| **$\text{Branch Taken}$** | $8$ |


### d)


In [24]:
calc_speedup("MinHaz", 118)


$\text{SpeedUp} = \frac{\text{CPU}_{{\text{Time}}_{\text{Base}}}}{\text{CPU}_{{\text{Time}}_{\text{Forwarding}}}} = \frac{\text{\# Cycles}_{\text{Base}}}{\text{\# Cycles}_{\text{Forwarding}}} = \frac{174}{118} \approx 1.475$

(simulated CPU Rate is consistent)

## 2.4


### a)


In [12]:
show_code("lab3_kit/2.4prog.s")


```                .data
A:              .word   1, 3, 1, 6, 4
                .word   2, 4, 3, 9, 5
mult:   .word   0

        .code
        daddi   $1, $0, A      ; *A[0]
        daddi   $5, $0, 1      ; $5 = 1 ;; i    
        daddi   $6, $0, 9      ; $6 = N - 1 ;; N = 10
        lw      $9, 0($1)      ; $9 = A[0]  ;; mult
        lw      $12, 8($1)     ; $12 = A[i]

loop:   dmul    $12, $12, $9   ; $12 = A[i]*mult
        daddi   $5, $5, 2      ; i += 2
        lw      $22, 16($1)    ; $12 = A[i+1]
        dadd    $9, $9, $12    ; mult = mult + A[i]*mult

        dmul    $22, $22, $9   ; $22 = A[i+1]*mult
        daddi   $1, $1, 16     ; 
        lw      $12, 8($1)     ; $12 = A[i+2]
        dadd    $9, $9, $22    ; mult = mult + A[i+1]*mult
        bne     $6, $5, loop   ; Exit loop if i == (N-1)

        ; 9 iters - missing 1
        dmul    $12, $12, $9   ; $12 = A[i]*mult
        dadd    $9, $9, $12    ; mult = mult + A[i]*mult
        
        sw      $9, mult($0)   ; Store result
        halt

;; Expected result: mult = f6180 (hex), 1008000 (dec)

```

### b)


![2.4](assets/2.4.png)


### c)


| **$\text{Clock Cycles}$** | $91$    |
| ------------------------- | ------- |
| **$\text{Instructions}$** | $45$    |
| **$\text{Average CPI}$**  | $2.022$ |

Stalls
| **$\text{Data}$** | $48$ |
| ----------------------------------|--------|
| **$\text{Structural}$** | $9$ |
| **$\text{Branch Taken}$** | $3$ |


### d)


In [27]:
calc_speedup("LUnrolling", 91)


$\text{SpeedUp} = \frac{\text{CPU}_{{\text{Time}}_{\text{Base}}}}{\text{CPU}_{{\text{Time}}_{\text{LUnrolling}}}} = \frac{\text{\# Cycles}_{\text{Base}}}{\text{\# Cycles}_{\text{LUnrolling}}} = \frac{174}{91} \approx 1.912$

(simulated CPU Rate is consistent)

## 2.5


### a)


In [15]:
show_code("lab3_kit/2.5prog.s")


```                .data
A:              .word   1, 3, 1, 6, 4
                .word   2, 4, 3, 9, 5
mult:   .word   0

        .code
        daddi   $1, $0, A      ; *A[0]
        daddi   $5, $0, 1      ; $5 = 1 ;; i
        daddi   $6, $0, 10     ; $6 = N ;; N = 10
        lw      $9, 0($1)      ; $9 = A[0]  ;; mult
        daddi   $1, $1, 8      ;

loop:   lw      $12, 0($1)     ; $12 = A[i]
        daddi   $5, $5, 1      ; i++
        dmul    $12, $12, $9   ; $12 = $12*$9 ;; $12 = A[i]*mult
        daddi   $1, $1, 8      ;

        bne     $6, $5, loop   ; Exit loop if i == N
        dadd    $9, $9, $12    ; $9 = $9 + $12  ;; mult = mult + A[i]*mult

        sw      $9, mult($0)   ; Store result
        halt

;; Expected result: mult = f6180 (hex), 1008000 (dec)```

### b)


![2.5](assets/2.5.png)


### d)


| **$\text{Clock Cycles}$** | $101$   |
| ------------------------- | ------- |
| **$\text{Instructions}$** | $61$    |
| **$\text{Average CPI}$**  | $1.656$ |

Stalls
| **$\text{Data}$** | $27$ |
| ----------------------------------|--------|
| **$\text{Structural}$** | $9$ |
| **$\text{Branch Taken}$** | $0$ |


### e)


In [26]:
calc_speedup("BDelay", 101)


$\text{SpeedUp} = \frac{\text{CPU}_{{\text{Time}}_{\text{Base}}}}{\text{CPU}_{{\text{Time}}_{\text{BDelay}}}} = \frac{\text{\# Cycles}_{\text{Base}}}{\text{\# Cycles}_{\text{BDelay}}} = \frac{174}{101} \approx 1.723$

(simulated CPU Rate is consistent)